# Multi-Agent System with Google ADK

This notebook demonstrates how to create a multi-agent system using Google's Agent Development Kit (ADK). The system consists of:

1. A Billing Agent that handles billing and payment-related inquiries
2. A Support Agent that provides technical support
3. A Coordinator Agent that routes user requests to the appropriate specialized agent

The notebook shows how to set up these agents, create a runner, and process user queries.

## Import Required Libraries

First, we need to import the necessary libraries from Google ADK and configure the API key.

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.genai import types
import google.generativeai as genai
import os
from google.adk.models.lite_llm import LiteLlm
import logging

# Configure the API key
genai.configure(api_key="")
os.environ['OPENAI_API_KEY'] = ''
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"
MODEL_GPT_4O = "gpt-3.5-turbo-0125"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger("<<MultiAgentTest>>")

## Define the Specialized Sub-Agents

We'll create two specialized agents:
1. A Billing Agent for handling payment-related inquiries
2. A Support Agent for handling technical support requests

In [2]:
# Define specialized sub-agents
billing_agent = LlmAgent(
    name="Billing",
    model=LiteLlm(model=MODEL_GPT_4O),
    instruction="You handle billing and payment-related inquiries.",
    description="Handles billing inquiries."
)

support_agent = LlmAgent(
    name="Support",
    model=LiteLlm(model=MODEL_GPT_4O),
    instruction="You provide technical support and troubleshooting assistance.",
    description="Handles technical support requests."
)

## Define the Coordinator Agent

Now we'll create a coordinator agent that will route user requests to the appropriate specialized agent.

In [3]:
# Define the coordinator agent
coordinator = LlmAgent(
    name="HelpDeskCoordinator",
    model=LiteLlm(model=MODEL_GPT_4O),
    instruction="Route user requests: Use Billing agent for payment issues, Support agent for technical problems.",
    description="Main help desk router.",
    sub_agents=[billing_agent, support_agent]
)

# For ADK compatibility, the root agent must be named `root_agent`
root_agent = coordinator

## Create a Runner

The Runner is responsible for executing the agent and managing the conversation flow.

In [4]:
runner = Runner(
        app_name="test_agent",
        agent=root_agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),)

## Process a User Query with the Multi-Agent System

Now let's simulate a user query and see how our multi-agent system handles it.

In [5]:
# Simulate a user query
user_query = "I can't log in gmail, help to give advice."
# user_query = "my billing is not working, help to give advice."
user_id = "test_user"
session_id = "test_session"

# Create a session
session = runner.session_service.create_session(
    app_name="test_agent",
    user_id=user_id,
    state={},
    session_id=session_id,
)

# Create a content object with the user query
content = types.Content(
    role="user", 
    parts=[types.Part.from_text(text=user_query)]
)

# Run the agent with the correct parameters
events = list(runner.run(
    user_id=user_id, 
    session_id=session.id, 
    new_message=content
))

# Process the events to get the response
response = ""
if events and events[-1].content and events[-1].content.parts:
    for event in events:
        logger.info(f"Event: {event.author}, Actions: {event.actions}")
        response = "\n".join([p.text for p in events[-1].content.parts if p.text])

print(response)

09:08:36 - LiteLLM:INFO: utils.py:2870 - 
LiteLLM completion() model= gpt-3.5-turbo-0125; provider = openai
2025-05-14 09:08:36,440 - LiteLLM - INFO - 
LiteLLM completion() model= gpt-3.5-turbo-0125; provider = openai
2025-05-14 09:08:36,994 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:08:37 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
2025-05-14 09:08:37,005 - LiteLLM - INFO - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
09:08:37 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
2025-05-14 09:08:37,010 - LiteLLM - INFO - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
09:08:37 - LiteLLM:INFO: utils.py:2870 - 
LiteLLM completion() model= gpt-3.5-turbo-0125; provider = openai
2025-05-14 09:08:37,011 - LiteLLM - INFO - 
LiteLLM completion() model= gpt-3.5-turbo-0125; pro

Hello! I can help you with that. When you are unable to log in to Gmail, there are a few troubleshooting steps you can try:

1. Double-check your username and password: Ensure that you are entering the correct email address and password. Pay attention to capitalization and special characters.

2. Clear your browser's cache and cookies: Sometimes stored data can cause login issues. Clearing the cache and cookies can help resolve this.

3. Try using a different browser: If you are having trouble logging in, try using a different web browser to see if the issue persists.

4. Check for any ongoing service outages: Sometimes Gmail may experience service disruptions. You can check the G Suite Status Dashboard for any reported issues.

If you have tried these steps and are still unable to log in, please let me know so we can explore further options.


## Try Different User Queries

You can modify the user query in the cell below to test how the system routes different types of requests.

In [6]:
# Try a different query
def process_query(query):
    # Create a new session for each query
    session_id = f"test_session_{hash(query)}"
    
    # Create a session
    session = runner.session_service.create_session(
        app_name="test_agent",
        user_id=user_id,
        state={},
        session_id=session_id,
    )
    
    # Create a content object with the user query
    content = types.Content(
        role="user", 
        parts=[types.Part.from_text(text=query)]
    )
    
    # Run the agent with the correct parameters
    events = list(runner.run(
        user_id=user_id, 
        session_id=session.id, 
        new_message=content
    ))
    
    # Process the events to get the response
    response = ""
    if events and events[-1].content and events[-1].content.parts:
        response = "\n".join([p.text for p in events[-1].content.parts if p.text])
    
    return response

# Example billing query
billing_query = "my billing is not working, help to give advice."
print(f"Query: {billing_query}")
print(f"Response: {process_query(billing_query)}")
print("\n" + "-"*50 + "\n")

# Example support query
support_query = "My application keeps crashing"
print(f"Query: {support_query}")
print(f"Response: {process_query(support_query)}")

09:08:42 - LiteLLM:INFO: utils.py:2870 - 
LiteLLM completion() model= gpt-3.5-turbo-0125; provider = openai
2025-05-14 09:08:42,146 - LiteLLM - INFO - 
LiteLLM completion() model= gpt-3.5-turbo-0125; provider = openai


Query: my billing is not working, help to give advice.


2025-05-14 09:08:42,752 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:08:42 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
2025-05-14 09:08:42,757 - LiteLLM - INFO - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
09:08:42 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
2025-05-14 09:08:42,765 - LiteLLM - INFO - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
09:08:42 - LiteLLM:INFO: utils.py:2870 - 
LiteLLM completion() model= gpt-3.5-turbo-0125; provider = openai
2025-05-14 09:08:42,765 - LiteLLM - INFO - 
LiteLLM completion() model= gpt-3.5-turbo-0125; provider = openai
2025-05-14 09:08:43,332 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:08:43 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost cal

Response: I am here to help with your billing issue. Please provide me with more details about the problem you are facing with your billing.

--------------------------------------------------

Query: My application keeps crashing


2025-05-14 09:08:44,099 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:08:44 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
2025-05-14 09:08:44,103 - LiteLLM - INFO - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
09:08:44 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
2025-05-14 09:08:44,107 - LiteLLM - INFO - selected model name for cost calculation: openai/gpt-3.5-turbo-0125
09:08:44 - LiteLLM:INFO: utils.py:2870 - 
LiteLLM completion() model= gpt-3.5-turbo-0125; provider = openai
2025-05-14 09:08:44,108 - LiteLLM - INFO - 
LiteLLM completion() model= gpt-3.5-turbo-0125; provider = openai
2025-05-14 09:08:44,761 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:08:44 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost cal

Response: I am here to help with the crashing issue of your application. Could you provide more details about when the crashes occur and any error messages you see?
